In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary
import torchvision.models as torch_models

import onnx
import onnxruntime

import io
import os
import json
import base64
import random
from PIL import Image

In [2]:
# dataset

dataset_path = '/home/wyundi/Server/Courses/CS546/project/data/cat/train'

## Read Data

In [3]:
# print(cat_df.iloc[0].at['image_base64_string'])

In [4]:
# for i in range(cat_df.shape[0]):
#     img = base64.b64decode(cat_df.iloc[i].at['image_base64_string'])
#     img = Image.open(io.BytesIO(img))
#     img_np = np.array(img)
#     print(img_np.shape) 

In [5]:
# img = base64.b64decode(cat_df.iloc[3].at['image_base64_string'])
# img = Image.open(io.BytesIO(img))
# plt.imshow(img)

# label = cat_df.iloc[3].at['label']
# print(label)

In [6]:
# img = img.resize((224, 224))
# plt.imshow(img)

In [7]:
# label = cat_df.iloc[0].at['label']
# print(label)
# print(type(label))

In [8]:
# img_np = np.array(img)
# print(img_np.shape)

# img_tensor = torch.tensor(img_np.transpose(2, 0, 1), dtype=torch.float32)
# print(img_tensor.size())

## Model

In [9]:
# device

def set_device(net, device='GPU'):
    if device == 'GPU':
        torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        torch_device = torch.device("cpu")

    if torch.cuda.device_count() > 1:
        model = nn.DataRarallel(model)

    net = net.to(torch_device)
    return net, torch_device

In [10]:
class dataset(Dataset):
    def __init__(self, path):
        
        self.__dataset_path = path
        
        self.__transforms = torch.nn.Sequential(
            T.Resize((224, 224)),
            T.RandomResizedCrop(224),
            T.RandomHorizontalFlip(),
        )
        
        self.data_list = []
        self.label_list = []
        
        for filename in os.listdir(path):
            self.__img_path = path + '/' + filename
            
            self.data_list.append(self.__img_path)
            self.label_list.append(0 if filename.split('.')[0] == 'cat' else 1)
            
        self.data = list(zip(self.data_list, self.label_list))
        
    def __getitem__(self, index):
        
        self.__img_path, self.__img_label = self.data[index]
        
        self.__img = Image.open(self.__img_path)
        self.__img = self.__transforms(self.__img)
        self.__img_np = np.array(self.__img)
        
        self.__img_label = torch.tensor(self.__img_label, dtype=torch.long)
        self.__img_tensor = torch.tensor(self.__img_np.transpose(2, 0, 1), dtype=torch.float32)
        
        self.__mean, self.__std = self.__img_tensor.mean(), self.__img_tensor.std()
        self.__std = 1e-03 if self.__std == 0 else self.__std
    
        self.__transforms_norm = torch.nn.Sequential(
            T.Normalize(self.__mean, self.__std)
        )

        self.__img_tensor = self.__transforms_norm(self.__img_tensor)
        
        return (self.__img_tensor, self.__img_label)
    
    def __len__(self):
        return len(self.data)

    def get_name(self):
        return self.name_list
        
cat_dataset = dataset(dataset_path)
dl = DataLoader(cat_dataset, batch_size=1, shuffle=True)

# for step, (inputs, labels) in enumerate(dl):
#     pass

In [11]:
class Convnet(nn.Module):
    def __init__(self, num_classes=1000):
        super(Convnet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten(start_dim=1, end_dim=3)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x




Net = Convnet(2)
input_shape = (16, 3, 224, 224)

print(Net)

print(summary(Net, input_shape))

Convnet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(s

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10,2)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [13]:
# helper function for classification

def calc_acc(out, labels):
    num = out.size(0)
    prediction = out.argmax(dim=1)
    return (prediction == labels).sum().item()/num

def evaluate(model, torch_device, loss_func, dataloader, method='classification'):
    loss = 0
    acc = 0

    loss_list = []
    acc_list = []
    
    with torch.no_grad():
        for step, (inputs, labels) in enumerate(dataloader):
            
            inputs = inputs.to(torch_device)
            labels = labels.to(torch_device)

            out = model(inputs)
            loss_list.append(loss_func(out, labels))

            if method == 'classification':
                acc_list.append(calc_acc(out, labels))

        loss = torch.mean(torch.tensor(loss_list))

    if method == 'classification':
        acc = torch.mean(torch.tensor(acc_list, dtype=torch.float32))
        return loss, acc
    elif method == 'regression':
        return loss


In [14]:
# Define a train function

def train_model(model, train_dataset, test_dataset, batch = 256, epochs=50,
                lr=0.005, class_weights = None, weight_decay = 0):

    # train_history
    train_history = {}
    train_history['train_loss'] = []
    train_history['train_acc'] = []

    train_history['test_loss'] = []
    train_history['test_acc'] = []

    # set device
    model, torch_device = set_device(model)
    
    if class_weights != None:
        class_weights = class_weights.to(torch_device)

    # Dataloader
    train_dl = DataLoader(train_dataset, batch_size=batch, shuffle=True, drop_last=True)
    test_dl = DataLoader(test_dataset, batch_size=batch, shuffle=True, drop_last=True)
    
    # optimzer and loss_func
    optimzer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
    loss_func = nn.CrossEntropyLoss(weight = class_weights)

    # train_process
    for epoch in range(epochs):
        
        for step, (inputs, labels) in enumerate(train_dl):
            
            inputs = inputs.to(torch_device)
            labels = labels.to(torch_device)

            out = model(inputs)
            loss = loss_func(out, labels)
            optimzer.zero_grad()
            loss.backward()
            optimzer.step()
            
            # output
            train_loss, train_acc = evaluate(model, torch_device, loss_func, train_dl)
            test_loss, test_acc = evaluate(model, torch_device, loss_func, test_dl)

            print(  'Epoch:', epoch+1, '/', epochs, ', '\
                    'train_loss: {loss:.5f}, '.format(loss = train_loss), \
                    'train_acc: {acc:.5f}, '.format(acc = train_acc), \
                    'test_loss: {loss:.5f}, '.format(loss = test_loss), \
                    'test_acc: {acc:.5f}'.format(acc = test_acc))

            train_history['train_loss'].append(train_loss)
            train_history['train_acc'].append(train_acc)

            train_history['test_loss'].append(test_loss)
            train_history['test_acc'].append(test_acc)
  
    return train_history

In [20]:
cat_dataset = dataset(dataset_path)

split_rate = 0.9
train_size = int(split_rate * len(cat_dataset))
test_size = len(cat_dataset) - train_size

print(test_size)

train_dataset, test_dataset = random_split(cat_dataset, [train_size, test_size])

batch = 512
epochs = 10
lr = 0.003

2500


In [ ]:
"""
Train Model
"""

# load model
# model = Convnet(2)
model = CNN()
model.load_state_dict(torch.load('/home/wyundi/Server/Courses/CS546/project/repo/M_Hub/src/ipynb/cat.pth'))
print(summary(model, (batch, 3, 224, 224)))

# train
hist = train_model(model, train_dataset, test_dataset, batch, epochs, lr)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Sequential: 1-1                        [512, 16, 55, 55]         --
│    └─Conv2d: 2-1                       [512, 16, 111, 111]       448
│    └─BatchNorm2d: 2-2                  [512, 16, 111, 111]       32
│    └─ReLU: 2-3                         [512, 16, 111, 111]       --
│    └─MaxPool2d: 2-4                    [512, 16, 55, 55]         --
├─Sequential: 1-2                        [512, 32, 13, 13]         --
│    └─Conv2d: 2-5                       [512, 32, 27, 27]         4,640
│    └─BatchNorm2d: 2-6                  [512, 32, 27, 27]         64
│    └─ReLU: 2-7                         [512, 32, 27, 27]         --
│    └─MaxPool2d: 2-8                    [512, 32, 13, 13]         --
├─Sequential: 1-3                        [512, 64, 3, 3]           --
│    └─Conv2d: 2-9                       [512, 64, 6, 6]           18,496
│    └─

In [17]:
save_path = '/home/wyundi/Server/Courses/CS546/project/repo/M_Hub/src/ipynb/cat.pth'
torch.save(model.state_dict(), save_path)

## Export model to ONNX